# Table of content : 

# 1. Problem statment : 

The goal is to develop a Machine Learning model that accurately predicts the price of a car based on various features such as brand, condition, mileage, fuel type, transmission, year, and other relevant attributes. 

This model aims to assist users in estimating fair car prices and making informed buying or selling decisions.

# 2. Data collection : 

The dataset was collected through web scraping from two popular Moroccan car listing platforms: `moteur.ma` and `avito.ma`, ensuring a 
diverse and representative sample of used cars available in the market.

# 3. Data undrestanding :

### Inisialize spark session : 

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark Prediction model1").getOrCreate()

# Check if the Spark session is created
print(spark.version)

3.5.5


### Load the Data : 

In [3]:
# Read csv file into pyspark dataFrame : 
raw_data = spark.read.csv("cleaned_data.csv",  header=True, inferSchema=True)

We have a problem with the categorical columns where "\NaN\" which is N/A is considered a category not a null value . Hence we should transform it to null using a python function .

In [4]:
from pyspark.sql.functions import col, when, regexp_replace, lit
from pyspark.sql.types import StringType

def transform_spark(df):
    for column, dtype in df.dtypes:
        if dtype == 'string':
            df = df.withColumn(
                column,
                when(
                    col(column).rlike(r'(?i)\\"nan\\"'),
                    None
                ).otherwise(col(column))
            )
    return df

In [5]:
# Apply the Transform function
data = transform_spark(raw_data)

In [6]:
# For better visualisation : 
data.limit(5).toPandas()

,id,brand,condition,creator,door_count,equipment,first_owner,fiscal_power,fuel_type,image_folder,...,model,origin,price,publication_date,sector,seller_city,source,title,transmission,year
0,9b006bc1-dc96-4dc0-bdd2-0c31a28d555a,mercedes-benz,None,MOHAMED,NaN,"Jantes aluminium, Alarme, Airbags, Abs, Vitres...",None,10,diesel,417803_MERCEDES-BENZ_250,...,250,None,51000.0,None,Ben Guerir,Benguerir,moteur,MERCEDES-BENZ 250,Manuelle,1998
1,e851563f-cac5-4703-9f50-26d10d59b89e,mitsubishi,Très bon,Timlalin Dome,3.0,"Airbags, Caméra de recul, Climatisation, ESP, ...",Non,11,diesel,3449_Mitsubishi_Pajero,...,Pajero,WW au Maroc,85000.0,11/02/2025 00:00,Agadir,Autre secteur,avito,Mitsubishi Pajero,Manuelle,2007
2,43e6114c-7df9-4ebf-a611-4153e866324f,fiat,Excellent,Louvre AutoShopBadge Icon,5.0,"CD/MP3/Bluetooth, Jantes aluminium",Oui,6,diesel,3360_Fiat_Doblo_Modèle,...,Doblo,WW au Maroc,NaN,22/03/2025 00:00,Casablanca,Maarif,avito,Fiat Doblo Modèle,Manuelle,2021
3,dd4a01b6-0cae-4e28-8c0f-28ee6844a823,land rover,Excellent,imad elmajdoub,5.0,"ABS, Airbags, CD/MP3/Bluetooth, Caméra de recu...",Oui,8,diesel,2694_لاند_روڨر_ديسكوفري_ديزل_أوتوماتيك_2017_في...,...,Discovery,WW au Maroc,190000.0,14/09/2024 00:00,Agadir,Haut Anza,avito,لاند روڨر ديسكوفري ديزل أوتوماتيك 2017 في أكادير,Automatique,2017
4,8c94b460-b7a4-47ba-b84c-a2e146d0e5ff,dacia,Excellent,ayoub,5.0,"ABS, Airbags, CD/MP3/Bluetooth, Caméra de recu...",Oui,6,diesel,700_dacia_duster,...,Duster,WW au Maroc,167000.0,11/02/2025 00:00,Casablanca,Sidi Bernoussi,avito,dacia duster,Manuelle,2019


### Drop columns : 

In [7]:
data = data.drop('image_folder')
data = data.drop('id')
data = data.drop('creator')
data = data.drop('title')
data = data.drop('source')

In [8]:
data.limit(3).toPandas()

,brand,condition,door_count,equipment,first_owner,fiscal_power,fuel_type,mileage,model,origin,price,publication_date,sector,seller_city,transmission,year
0,mercedes-benz,None,NaN,"Jantes aluminium, Alarme, Airbags, Abs, Vitres...",None,10,diesel,65000,250,None,51000.0,None,Ben Guerir,Benguerir,Manuelle,1998
1,mitsubishi,Très bon,3.0,"Airbags, Caméra de recul, Climatisation, ESP, ...",Non,11,diesel,274999,Pajero,WW au Maroc,85000.0,11/02/2025 00:00,Agadir,Autre secteur,Manuelle,2007
2,fiat,Excellent,5.0,"CD/MP3/Bluetooth, Jantes aluminium",Oui,6,diesel,104999,Doblo,WW au Maroc,NaN,22/03/2025 00:00,Casablanca,Maarif,Manuelle,2021


### Data Size : 

In [9]:
# Number of rows
num_rows = data.count()

# Number of columns
num_columns = len(data.columns)

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 60264
Number of columns: 16


### Get data schema :

In [10]:
# Print schema of the data
data.printSchema()

root
 |-- brand: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- door_count: integer (nullable = true)
 |-- equipment: string (nullable = true)
 |-- first_owner: string (nullable = true)
 |-- fiscal_power: integer (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- mileage: integer (nullable = true)
 |-- model: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- publication_date: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- year: integer (nullable = true)



### Categorical columns VS numerical columns : 

| Categorical   | numerical      |
|---------------|----------------|
| condition     | fiscal_power   |
| transmission  | price (target) |
| equipment     | publication_date|
| first_owner   | year           |
| fuel_type     | door_count     |
| model         | mileage        |
| origin        |                |
| seller_city   |                |
| brand         |                |
| sector        |                 
| source        |


______________

# 4.Exploratory Data Analysis (EDA) :

### Unique values for each column :

In [12]:
# Show unique values for each column :
for col in data.columns:
    print(f"Column: {col}")
    data.select(col).distinct().show(truncate=False)

Column: brand
+----------+
|brand     |
+----------+
|citroën   |
|chery     |
|jaguar    |
|changhe   |
|autres    |
|land rover|
|ktm       |
|daihatsu  |
|mahindra  |
|hummer    |
|tata      |
|autre     |
|mitsubishi|
|land-rover|
|lexus     |
|lada      |
|toyota    |
|seat      |
|cupra     |
|peugeot   |
+----------+
only showing top 20 rows

Column: condition
+-----------+
|condition  |
+-----------+
|Très bon   |
|Excellent  |
|Correct    |
|Neuf       |
|Pour Pièces|
|Endommagé  |
|Bon        |
|NULL       |
+-----------+

Column: door_count
+----------+
|door_count|
+----------+
|3         |
|5         |
|4         |
|2         |
|NULL      |
+----------+

Column: equipment
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Statistics summary for numerical variables : 

In [ ]:
numerical_columns = [ 'fiscal_power' , 'price' , 'year' , 'door_count' , 'mileage' ]
data.describe(numerical_columns).toPandas()

____________

# 5. Data preprocessing : 

### Check for missing values : 

In [13]:
from pyspark.sql import functions as F

# Counting missing (null) values in each column
data.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in data.columns]).toPandas()

,brand,condition,door_count,equipment,first_owner,fiscal_power,fuel_type,mileage,model,origin,price,publication_date,sector,seller_city,transmission,year
0,1506,26093,9159,20308,20575,2957,0,3134,1506,28776,12674,23209,2,1505,0,2806


### Drop missing values : 

In [14]:
# Drop all the missing values :
data_cleaned = data.dropna()

In [15]:
from pyspark.sql import functions as F

# Counting missing (null) values in each column
data_cleaned.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in data_cleaned.columns]).toPandas()

,brand,condition,door_count,equipment,first_owner,fiscal_power,fuel_type,mileage,model,origin,price,publication_date,sector,seller_city,transmission,year
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
# Data size (rows) before and after : 
print(f"Data size before droping missing values : {data.count()}")
print(f"Data size after droping missing values : {data_cleaned.count()}")

Data size before droping missing values : 60264
Data size after droping missing values : 14416


## Features ingineering :

### Split data_publication column : 

In [23]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType


data_cleaned = data_cleaned.withColumn("publication_date", F.to_timestamp("publication_date", "dd/MM/yyyy HH:mm"))

# Extract Year, Month, Day, Weekday, Is_Weekend, and Days_since_posted
data_features = data_cleaned.withColumn("publication_Year", F.year("publication_date")) \
       .withColumn("publication_Month", F.month("publication_date")) \
       .withColumn("publication_Day", F.dayofmonth("publication_date")) \
       .withColumn("publication_Weekday", F.dayofweek("publication_date")) \
       .withColumn("Is_Weekend", (F.dayofweek("publication_date") >= 6).cast(IntegerType())) \
       .withColumn("Days_since_posted", F.datediff(F.current_date(), "publication_date"))


In [24]:
# Show the result
data_features.select("publication_date", "publication_Year", "publication_Month", "publication_Day", "publication_Weekday", "Is_Weekend", "Days_since_posted").show(5)

+-------------------+----------------+-----------------+---------------+-------------------+----------+-----------------+
|   publication_date|publication_Year|publication_Month|publication_Day|publication_Weekday|Is_Weekend|Days_since_posted|
+-------------------+----------------+-----------------+---------------+-------------------+----------+-----------------+
|2025-02-11 00:00:00|            2025|                2|             11|                  3|         0|               77|
|2024-09-14 00:00:00|            2024|                9|             14|                  7|         1|              227|
|2025-02-11 00:00:00|            2025|                2|             11|                  3|         0|               77|
|2024-10-14 00:00:00|            2024|               10|             14|                  2|         0|              197|
|2025-02-11 00:00:00|            2025|                2|             11|                  3|         0|               77|
+-------------------+---

In [20]:
data_features.limit(5).toPandas()

,brand,condition,door_count,equipment,first_owner,fiscal_power,fuel_type,mileage,model,origin,...,sector,seller_city,transmission,year,publication_Year,publication_Month,publication_Day,publication_Weekday,Is_Weekend,Days_since_posted
0,mitsubishi,Très bon,3,"Airbags, Caméra de recul, Climatisation, ESP, ...",Non,11,diesel,274999,Pajero,WW au Maroc,...,Agadir,Autre secteur,Manuelle,2007,2025,2,11,3,0,77
1,land rover,Excellent,5,"ABS, Airbags, CD/MP3/Bluetooth, Caméra de recu...",Oui,8,diesel,22499,Discovery,WW au Maroc,...,Agadir,Haut Anza,Automatique,2017,2024,9,14,7,1,227
2,dacia,Excellent,5,"ABS, Airbags, CD/MP3/Bluetooth, Caméra de recu...",Oui,6,diesel,72499,Duster,WW au Maroc,...,Casablanca,Sidi Bernoussi,Manuelle,2019,2025,2,11,3,0,77
3,audi,Excellent,5,"ABS, Airbags, CD/MP3/Bluetooth, Climatisation,...",Non,11,diesel,184999,Q5,WW au Maroc,...,Casablanca,Ain Sebaa,Automatique,2010,2024,10,14,2,0,197
4,volkswagen,Excellent,5,"ABS, CD/MP3/Bluetooth, Caméra de recul, Climat...",Non,12,diesel,274999,Touareg,Dédouanée,...,Oujda,Boustane,Automatique,2006,2025,2,11,3,0,77


#### Drop publication_date : 

In [26]:
data_features = data_features.drop("publication_date")

### Split equipments column :

In [31]:
data_features.select("equipment").limit(5).toPandas()

,equipment
0,"Airbags, Caméra de recul, Climatisation, ESP, ..."
1,"ABS, Airbags, CD/MP3/Bluetooth, Caméra de recu..."
2,"ABS, Airbags, CD/MP3/Bluetooth, Caméra de recu..."
3,"ABS, Airbags, CD/MP3/Bluetooth, Climatisation,..."
4,"ABS, CD/MP3/Bluetooth, Caméra de recul, Climat..."


In [28]:
# Declare different equipments types : 
equipment_types = [
    "aluminum_rims", "airbags", "air_conditioning", "navigation_system",
    "sunroof", "leather_seats", "parking_sensor", "rear_camera",
    "electric_windows", "abs", "esp", "cruise_control", 
    "speed_limiter", "cd_mp3_bluetooth", "onboard_computer", "central_locking"
]

In [32]:
from pyspark.sql.functions import col, when, lower, lit

def encode_equipment(df, equipment_col="equipment"):
    for eq in equipment_types:
        # Generate a clean column name
        new_col = eq.lower().replace(" ", "_").replace("/", "_").replace("-", "_")
        # Add column with True/False if the equipment exists
        df = df.withColumn(
            new_col,
            when(lower(col(equipment_col)).contains(eq.lower()), lit(True)).otherwise(lit(False))
        )
    return df


In [33]:
data_equipment = encode_equipment(data_features)

In [35]:
data_equipment.select("equipment", "airbags", "abs", "esp", "caméra_de_recul").show(5)

+--------------------+-------+-----+----+---------------+
|           equipment|airbags|  abs| esp|caméra_de_recul|
+--------------------+-------+-----+----+---------------+
|Airbags, Caméra d...|   true|false|true|           true|
|ABS, Airbags, CD/...|   true| true|true|           true|
|ABS, Airbags, CD/...|   true| true|true|           true|
|ABS, Airbags, CD/...|   true| true|true|          false|
|ABS, CD/MP3/Bluet...|  false| true|true|           true|
+--------------------+-------+-----+----+---------------+
only showing top 5 rows



In [36]:
data_equipment.limit(6).toPandas()

,brand,condition,door_count,equipment,first_owner,fiscal_power,fuel_type,mileage,model,origin,...,radar_de_recul,caméra_de_recul,vitres_électriques,abs,esp,régulateur_de_vitesse,limiteur_de_vitesse,cd_mp3_bluetooth,ordinateur_de_bord,verrouillage_centralisé
0,mitsubishi,Très bon,3,"Airbags, Caméra de recul, Climatisation, ESP, ...",Non,11,diesel,274999,Pajero,WW au Maroc,...,True,True,True,False,True,False,False,False,True,True
1,land rover,Excellent,5,"ABS, Airbags, CD/MP3/Bluetooth, Caméra de recu...",Oui,8,diesel,22499,Discovery,WW au Maroc,...,True,True,True,True,True,True,True,True,True,True
2,dacia,Excellent,5,"ABS, Airbags, CD/MP3/Bluetooth, Caméra de recu...",Oui,6,diesel,72499,Duster,WW au Maroc,...,True,True,True,True,True,True,True,True,True,True
3,audi,Excellent,5,"ABS, Airbags, CD/MP3/Bluetooth, Climatisation,...",Non,11,diesel,184999,Q5,WW au Maroc,...,True,False,True,True,True,True,True,True,True,True
4,volkswagen,Excellent,5,"ABS, CD/MP3/Bluetooth, Caméra de recul, Climat...",Non,12,diesel,274999,Touareg,Dédouanée,...,True,True,True,True,True,True,True,True,True,True
5,peugeot,Excellent,5,"ABS, Airbags, CD/MP3/Bluetooth, Climatisation,...",Non,6,diesel,62499,208,WW au Maroc,...,True,False,True,True,True,True,True,True,True,True


###### Verification :

In [40]:
# Convert to Pandas DataFrame
pandas_df = data_equipment.toPandas()

# Save as CSV with Pandas
pandas_df.to_csv("verify_equipment1.csv", index=False)


#### Drop equipment column : 

In [44]:
data_equipment = data_equipment.drop("equipment")

### Data preprocessed 1 : 

In [47]:
data_equipment.limit(5).toPandas()

,brand,condition,door_count,first_owner,fiscal_power,fuel_type,mileage,model,origin,price,...,radar_de_recul,caméra_de_recul,vitres_électriques,abs,esp,régulateur_de_vitesse,limiteur_de_vitesse,cd_mp3_bluetooth,ordinateur_de_bord,verrouillage_centralisé
0,mitsubishi,Très bon,3,Non,11,diesel,274999,Pajero,WW au Maroc,85000,...,True,True,True,False,True,False,False,False,True,True
1,land rover,Excellent,5,Oui,8,diesel,22499,Discovery,WW au Maroc,190000,...,True,True,True,True,True,True,True,True,True,True
2,dacia,Excellent,5,Oui,6,diesel,72499,Duster,WW au Maroc,167000,...,True,True,True,True,True,True,True,True,True,True
3,audi,Excellent,5,Non,11,diesel,184999,Q5,WW au Maroc,120000,...,True,False,True,True,True,True,True,True,True,True
4,volkswagen,Excellent,5,Non,12,diesel,274999,Touareg,Dédouanée,70000,...,True,True,True,True,True,True,True,True,True,True


##### save to csv : 

In [48]:
# Save to csv : 
# Convert to Pandas DataFrame
pandas_df_df = data_equipment.toPandas()

# Save as CSV with Pandas
pandas_df_df.to_csv("data_preprocessed1.csv", index=False)

## Variable encoding : 

_______________

# 6. Model training :